In [1]:
import warnings
warnings.filterwarnings("ignore")
from keras.engine.topology import Layer
from keras.layers import Bidirectional, Concatenate, Permute, Dot, Input, LSTM, GRU, Multiply, Reshape
from keras.layers import RepeatVector, Dense, Activation, Lambda, Embedding
from keras.optimizers import Adam
from keras.utils import to_categorical
from keras.models import load_model, Model
import keras.backend as K
import keras
import numpy as np
import random
import tqdm
import matplotlib.pyplot as plt
from keras.utils import to_categorical
import pandas as pd
# from tcn import TCN
%matplotlib inline
from keras.layers import merge, Input, Dense, TimeDistributed, Lambda                                   
import keras
from keras.models import Sequential, Model
from keras.layers import Conv2D, GlobalMaxPooling2D,MaxPooling2D, Flatten, Dense, BatchNormalization, Activation, Input, GlobalAveragePooling2D,AveragePooling2D, Add
from keras.optimizers import Adam, rmsprop
from keras.callbacks import EarlyStopping
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dropout
from keras.wrappers.scikit_learn import KerasRegressor
from keras.constraints import maxnorm
MAX_SENT_LENGTH = 55
MAX_SENTS = 24
VALIDATION_SPLIT = 0.2

Using TensorFlow backend.


In [2]:
from sklearn.preprocessing import MinMaxScaler  
from sklearn.preprocessing import StandardScaler  
from pandas import concat

std = StandardScaler()

In [3]:
import pandas as pd
#matplotlib inline
#参数初始化
discfile = '../data/data_wea_time_unify.csv'

data = pd.read_csv(discfile)

In [4]:
import h5py
import os

ground = h5py.File('../data/inte_g_12_linear.h5', 'r')
g = ground['X']

ele_zs = h5py.File('../data/inte_e_zs_12_new_linear.h5', 'r')
ezs = ele_zs['X']

ele_sd = h5py.File('../data/inte_e_sd_12_new_linear.h5', 'r')
esd = ele_sd['X']

In [5]:
g_m=[g,ezs,esd]

In [6]:
g_m=np.array(g_m).reshape(6064,16,16,3)
g=np.array(g).reshape(6064,16,16,1)

In [ ]:
# from attention_keras import *
from keras.layers import LeakyReLU
class AttentionLayer(Layer):
    def __init__(self, **kwargs):
        super(AttentionLayer, self).__init__(** kwargs)
    
    def build(self, input_shape):
        assert len(input_shape)==3
        # W.shape = (time_steps, time_steps)
        self.W = self.add_weight(name='att_weight', 
                                 shape=(input_shape[1], input_shape[1]),
                                 initializer='uniform',
                                 trainable=True)
        super(AttentionLayer, self).build(input_shape)

    def call(self, inputs, mask=None):
        x = K.permute_dimensions(inputs, (0, 2, 1))
        # x.shape = (batch_size, seq_len, time_steps)
        # general
        a = K.softmax(K.tanh(K.dot(x, self.W)))
        a = K.permute_dimensions(a, (0, 2, 1))
        outputs = a * inputs
        outputs = K.sum(outputs, axis=1)
        return outputs

    def compute_output_shape(self, input_shape):
        return input_shape[0], input_shape[2]

def create_model(learn_rate=0.001, neurons=24, traffic_n=3):    
#     交通指数数据模块
    inp=Input(shape=(16, 16, 1))
    out=Conv2D(filters=1, kernel_size=3, strides=1)(out)
    out=MaxPooling2D(pool_size=(3, 3), strides=2)(out)
    out=Flatten()(out)

    out=Dense(3)(out)
    out=Dense(1)(out)
    tra=Model(inputs=inp, outputs=out)
    
#     特征组件
    sentence_input = Input(shape=(54+traffic_n,))
    # s_input=keras.layers.concatenate([sentence_input])
    embed = Reshape((54+traffic_n,1))(sentence_input)
    l_lstm=MAttention(8,16)([embed,embed,embed])
    l_dense = TimeDistributed(Dense(32))(l_lstm)

    l_att = AttentionLayer()(l_dense)

    sentEncoder = Model(sentence_input, l_att)

    
    review_input = Input(shape=(MAX_SENTS,54))
    r_input = Input(shape=(MAX_SENTS,16, 16, 3))
    r_encoder = TimeDistributed(tra)(r_input)
    s_input=keras.layers.concatenate([r_encoder,review_input])
    
    review_encoder = TimeDistributed(sentEncoder)(s_input)
    w_input=Input(shape=(MAX_SENTS,3))
    w_encoder = TimeDistributed(wm)(w_input)

    wea_input=Input(shape=(MAX_SENTS,5))
    wea_encoder = TimeDistributed(weam)(wea_input)
    l_lstm_sent=TCN(padding='same',return_sequences=True)(review_encoder)
    we_time_encoder=keras.layers.concatenate([w_encoder,wea_input,l_lstm_sent])

    l_dense_sent = TimeDistributed(Dense(64))(we_time_encoder)
    l_att_sent = AttentionLayer()(l_dense_sent)
    preds = Dense(futrue_time)(l_att_sent)
    model_1 = Model([w_input,wea_input,r_input,review_input], preds)
    model_1.compile(optimizer=Adam(lr=learn_rate, beta_1=0.99, beta_2=0.999, decay=0.0006),
                        loss='mae')
    return model_1

model=create_model()
history=model.fit([w_m,wea_m,ge_m,sd_m],td, epochs=20, batch_size=68,verbose=2,validation_split=0.2,shuffle=True)
# 绘制训练 & 验证的损失值
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()